Upload data to huggingface dataset

In [1]:
import pandas as pd
import os
import random
import numpy as np
import json
from datasets import DatasetDict, Dataset, Features, Value
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from datasets import load_dataset, DatasetDict
from huggingface_hub import HfApi

/home/ubuntu/miniforge3/envs/multimodal/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Convert the json file to csv format

In [2]:
dir = '../Financial_Alpha_Vantage/Summary_with_price_v0.2/'
data_dict = {}

# Iterate through each JSON file in the directory
for company in os.listdir(dir):
    json_dir = os.path.join(dir, company)

# Get list of JSON files, sorted by filename (assuming they are named in chronological order)
    json_files = sorted(glob(os.path.join(json_dir, '*.json')))

    # List to store input and output data
    data = []

    # Iterate through JSON files
    for i in range(len(json_files) - 1):
        # Read current JSON file
        with open(json_files[i], 'r') as current_file:
            current_data = json.load(current_file)
        
        # Read next JSON file for the summary key
        with open(json_files[i + 1], 'r') as next_file:
            next_data = json.load(next_file)
        
        # Extract input and output data
        input_data = current_data  # Convert JSON to string
        input_price = {'shareprice': current_data.get('share_price', '')}
        output_summary_price = {'summary': next_data.get('summary', ''), 'shareprice': next_data.get('share_price', '')}  # Get summary from next day's JSON
        output_price = {'shareprice': next_data.get('share_price', '')}
        
        # Append to data list
        data.append({
            'input': input_data,
            'input_price': input_price,
            'output_summary_price': output_summary_price,
            'output_price': output_price
        })

    # Convert data list to DataFrame
    df = pd.DataFrame(data)

    print(len(df))

    # Save DataFrame to CSV
    output_csv_path = f"../Dataset/Finance/v0.2/{company}.csv"
    df.to_csv(output_csv_path, index=False)
    

161


In [3]:
def split_data(file_path, train_ratio=0.8, validation_ratio=0.1, test_ratio=0.1):
    # Read the CSV file
    data = pd.read_csv(file_path)
    
    # Calculate the validation and test sizes
    val_size = validation_ratio / (test_ratio + validation_ratio)
    
    # Split the data into train and temporary datasets
    train_data, temp_data = train_test_split(data, test_size=(1 - train_ratio), random_state=42, shuffle=False)
    
    # Split the temporary dataset into validation and test datasets
    validation_data, test_data = train_test_split(temp_data, test_size=val_size, random_state=42, shuffle=False)
    
    # Save the datasets
    
    dir = file_path.split('/')[:-1]
    dir = '/'.join(dir)
    file_name = file_path.split('/')[-1]
    train_data.to_csv(f'{dir}/train_{file_name}', index=False)
    validation_data.to_csv(f'{dir}/val_{file_name}', index=False)
    test_data.to_csv(f'{dir}/test_{file_name}', index=False)

# List of your CSV files
dir = '../Dataset/Finance/v0.2'

# Loop through each file and split the data
for filename in os.listdir(dir):
    if not filename.startswith("train") and not filename.startswith("test") and not filename.startswith("val"):
        path = os.path.join(dir, filename)
        split_data(path)  # Adjust the directory path as needed

In [4]:
def convert_to_parquet(dataframe_test, dataframe_train, dataframe_val):
    train = pd.concat(dataframe_train)
    test = pd.concat(dataframe_test)
    val = pd.concat(dataframe_val)

    train_path = '../parquet_dir/train_finance.parquet'
    test_path = '../parquet_dir/test_finance.parquet'
    val_path = '../parquet_dir/val_finance.parquet'

    train.to_parquet(train_path, engine='pyarrow')
    test.to_parquet(test_path, engine='pyarrow')
    val.to_parquet(val_path, engine='pyarrow')
    # Load the dataset
    train_dataset = load_dataset('parquet', data_files=train_path, split='train')
    test_dataset = load_dataset('parquet', data_files=test_path, split='train')
    val_dataset = load_dataset('parquet', data_files=val_path, split = 'train')
    dataset_dict = DatasetDict({
        'train': train_dataset,
        'validation': val_dataset,
        'test': test_dataset
    })

    return dataset_dict

In [5]:
def load_from_huggingface(dataset_path, case):
    dataset = load_dataset(dataset_path)

# Access the train split
    for split in ['train', 'validation', 'test']:
        train_dataset = dataset[split]

        # Convert the dataset to a Pandas DataFrame
        df = train_dataset.to_pandas()

        # Save the DataFrame to a CSV file
        df.to_csv(f"../Data/Finance/1_day/{case}/{split}_all.csv", index=False)

# Case1 number -> number


In [6]:
dir = '../Dataset/Finance/v0.2/'
# case 1: number -> number
dataframe_train = []
dataframe_test = []
dataframe_val = []


for filename in os.listdir(dir):
    if filename.startswith('train') or filename.startswith('test') or filename.startswith('val'):
        split = filename.split('_')[0]
        path = os.path.join(dir, filename)
        summaries = pd.read_csv(path)
        instruction = "Given the share price for the current day, please predict the shared price for next day."

        df = summaries[['input_price', 'output_price']].rename(columns={'input_price': 'input', 'output_price': 'output'})
        df['instruction'] = instruction
        # skip the first and last historical_size days
    
    if filename.startswith('test'):
        dataframe_test.append(df)
    if filename.startswith('train'):
        dataframe_train.append(df)
    if filename.startswith('val'):
        dataframe_val.append(df)

dataset_dict = convert_to_parquet(dataframe_test, dataframe_train, dataframe_val)

token = os.getenv("HF_TOKEN")
# Push the dataset to the Hugging Face Hub
dataset_dict.push_to_hub(f"Howard881010/finance-numerical", token=token)


# Load the dataset from Hugging Face Hub
load_from_huggingface("Howard881010/finance-numerical", "numerical")


Generating train split: 128 examples [00:00, 2189.17 examples/s]
Generating train split: 17 examples [00:00, 241.18 examples/s]
Generating train split: 16 examples [00:00, 305.66 examples/s]
Generating test split: 100%|██████████| 17/17 [00:00<00:00, 319.34 examples/s]


# Case 2 number + text => number

In [7]:
dir = '../Dataset/Finance/v0.2/'
# case 1: number -> number
dataframe_train = []
dataframe_test = []
dataframe_val = []


for filename in os.listdir(dir):
    if filename.startswith('train') or filename.startswith('test') or filename.startswith('val'):
        split = filename.split('_')[0]
        path = os.path.join(dir, filename)
        summaries = pd.read_csv(path)
        instruction = "Given the financial report and the share price for the current day, please predict the shared price for next day."

        df = summaries[['input', 'output_price']].rename(columns={'output_price': 'output'})
        df['instruction'] = instruction
        # skip the first and last historical_size days
    
    if filename.startswith('test'):
        dataframe_test.append(df)
    if filename.startswith('train'):
        dataframe_train.append(df)
    if filename.startswith('val'):
        dataframe_val.append(df)

dataset_dict = convert_to_parquet(dataframe_test, dataframe_train, dataframe_val)

token = os.getenv("HF_TOKEN")
# Push the dataset to the Hugging Face Hub
dataset_dict.push_to_hub(f"Howard881010/finance-mixed-numerical", token=token)


# Load the dataset from Hugging Face Hub
load_from_huggingface("Howard881010/finance-mixed-numerical", "mixed-numerical")

Generating train split: 128 examples [00:00, 1374.80 examples/s]
Generating train split: 17 examples [00:00, 300.91 examples/s]
Generating train split: 16 examples [00:00, 226.62 examples/s]
Generating test split: 100%|██████████| 17/17 [00:00<00:00, 258.53 examples/s]


# Case 3 number + text => number + summary

In [8]:
dir = '../Dataset/Finance/v0.2/'
# case 1: number -> number
dataframe_train = []
dataframe_test = []
dataframe_val = []


for filename in os.listdir(dir):
    if filename.startswith('train') or filename.startswith('test') or filename.startswith('val'):
        split = filename.split('_')[0]
        path = os.path.join(dir, filename)
        summaries = pd.read_csv(path)
        instruction = "Given the financial report and share price for the current day, please predict the summary and shared price part for next day."

        df = summaries[['input', 'output_summary_price']].rename(columns={'output_summary_price': 'output'})
        df['instruction'] = instruction
        # skip the first and last historical_size days
    
    if filename.startswith('test'):
        dataframe_test.append(df)
    if filename.startswith('train'):
        dataframe_train.append(df)
    if filename.startswith('val'):
        dataframe_val.append(df)

dataset_dict = convert_to_parquet(dataframe_test, dataframe_train, dataframe_val)

token = os.getenv("HF_TOKEN")
# Push the dataset to the Hugging Face Hub
dataset_dict.push_to_hub(f"Howard881010/finance-mixed-summary", token=token)

# Load the dataset from Hugging Face Hub
load_from_huggingface("Howard881010/finance-mixed-summary", "mixed-summary")

Generating train split: 128 examples [00:00, 1623.43 examples/s]
Generating train split: 17 examples [00:00, 214.80 examples/s]
Generating train split: 16 examples [00:00, 231.73 examples/s]
Generating test split: 100%|██████████| 17/17 [00:00<00:00, 243.55 examples/s]
